In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup as bs
from pprint import pprint
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
import time
from time import sleep
import re 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


In [2]:
df = pd.read_csv('oscar_left_update_complete.csv')

In [3]:
print(df.shape)
df.head(20)

(925, 20)


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Film,Budget,Worldwide,Domestic,Release,Duration,MPAA.Rating,IMDB.Rating,Voter.Numbers,Metascore,Genre,Language,Opening.Weekend,Year,Award,Nomination,name
0,0,0,0,All Quiet on the Western Front,NaN,NaN,NaN,NaN,2h 32m,Passed,8.1,NaN,91.0,Drama,NaN,NaN,2022,4,9,all quiet on the western front
1,1,1,1,The Whale,NaN,"$54,697,540","$17,463,630",NaN,1h 57m,R,7.7,NaN,60.0,Drama,NaN,"$332,152",2022,2,3,the whale
2,2,2,2,Black Panther: Wakanda Forever,NaN,"$859,208,836","$453,829,060",NaN,2h 41m,PG-13,6.7,NaN,67.0,Action,NaN,"$181,339,761",2022,1,5,black panther: wakanda forever
3,3,3,3,Avatar: The Way of Water,NaN,"$2,320,250,281","$684,075,767",NaN,3h 12m,PG-13,7.7,NaN,67.0,Action,NaN,"$134,100,226",2022,1,4,avatar: the way of water
4,4,4,4,Women Talking,NaN,"$9,140,282","$5,456,531",NaN,1h 44m,PG-13,6.9,NaN,79.0,Drama,NaN,"$40,530",2022,1,2,women talking
5,5,5,5,Navalny,NaN,NaN,NaN,NaN,1h 39m,R,7.7,NaN,82.0,Documentary,NaN,NaN,2022,1,1,navalny
6,6,6,6,The Elephant Whisperers,NaN,"$26,023,860","$26,010,864",NaN,41m,PG,7.5,NaN,NaN,Documentary,NaN,NaN,2022,1,1,the elephant whisperers
7,7,7,7,An Irish Goodbye,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022,1,1,an irish goodbye
8,8,8,8,"The Boy, the Mole, the Fox and the Horse",NaN,"$156,612","$156,612",NaN,32m,TV-G,7.8,NaN,75.0,Animation,NaN,"$6,554",2022,1,1,"the boy, the mole, the fox and the horse"
9,9,9,9,RRR,NaN,"$166,611,197","$15,156,051",NaN,3h 7m,Not Rated,7.8,NaN,83.0,Action,NaN,"$9,500,000",2022,1,1,rrr


### Open Chrome Window

In [23]:
wd = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\redoc\AppData\Local\Temp\ipykernel_3780\3832095755.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


In [24]:
wd.get('https://www.wikipedia.com/')

## Go to Search Bar

In [25]:
search_movie = wd.find_element("xpath", "//*[@id='searchInput']") # //*[@id="search-word"]

In [26]:
search_movie.send_keys("Another Round"+" (film)")

In [27]:
search_movie.send_keys(Keys.RETURN)

In [28]:
infobox_table = wd.find_element("xpath", "//table[@class='infobox vevent']")

In [29]:
rows = infobox_table.find_elements(By.TAG_NAME, "tr")

In [9]:
budget_row = None
for row in rows:
    if "Budget" in row.text:
        budget_row = row
        break

if budget_row is not None:
    budget = budget_row.find_element(By.TAG_NAME, "td").text
    print("Movie budget:", budget)
else:
    print("Budget information not found.")

Movie budget: kr33.5 million[3]
(US$5.2 million)


In [30]:
release_row = None
for row in rows:
    if "Release" in row.text:
        release_row = row
        break

if release_row is not None:
    release = release_row.find_element(By.TAG_NAME, "td").text
    print("Movie budget:", release)
else:
    print("Budget information not found.")

Movie budget: 12 September 2020 (TIFF)
24 September 2020 (Denmark)


---

# Scrape from Wikipedia

In [39]:
df = pd.read_csv('oscar.csv')

wd = webdriver.Chrome(ChromeDriverManager().install())

movie_indices = np.arange(0,925,1)

for movie_index in movie_indices[450:925]:
    movie_title = df['Film'][movie_index] # get song title
    movie_year = df['Year'][movie_index] # get song title
    wd.get('https://www.wikipedia.com/') # go to main site
    search_movie = wd.find_element("xpath", "//*[@id='searchInput']") 
    search_movie.send_keys(str(movie_title)+" "+str(movie_year)+" film")
    search_movie.send_keys(Keys.RETURN)

    try:
        hyperlink = wd.find_element("xpath", "//*[@id='mw-content-text']/div[3]/div[4]/ul/li[1]/table/tbody/tr/td[2]/div[1]/a")
        hyperlink.send_keys(Keys.RETURN)
        infobox_table = wd.find_element("xpath", "//table[@class='infobox vevent']")
        rows = infobox_table.find_elements(By.TAG_NAME, "tr")
        #-----------
        budget_row = None
        for row in rows:
            if "Budget" in row.text:
                budget_row = row
                break

        if budget_row is not None:
            df['Budget'][movie_index] = budget_row.find_element(By.TAG_NAME, "td").text
        else:
            pass

        release_row = None
        for row in rows:
            if "Release" in row.text:
                release_row = row
                break
        #------------
        if release_row is not None:
            df['Release'][movie_index] = release_row.find_element(By.TAG_NAME, "td").text
        else:
            pass
    except:
        pass

C:\Users\redoc\AppData\Local\Temp\ipykernel_3780\839764468.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\redoc\AppData\Local\Temp\ipykernel_3780\839764468.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Release'][movie_index] = release_row.find_element(By.TAG_NAME, "td").text
C:\Users\redoc\AppData\Local\Temp\ipykernel_3780\839764468.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Budget'][movie_index] = budget_row.find_element(By.TAG_NAME, "td").text


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0078A813+48355]
	(No symbol) [0x0071C4B1]
	(No symbol) [0x00625358]
	(No symbol) [0x0060D293]
	(No symbol) [0x0066E37B]
	(No symbol) [0x0067C473]
	(No symbol) [0x0066A536]
	(No symbol) [0x006482DC]
	(No symbol) [0x006493DD]
	GetHandleVerifier [0x009EAABD+2539405]
	GetHandleVerifier [0x00A2A78F+2800735]
	GetHandleVerifier [0x00A2456C+2775612]
	GetHandleVerifier [0x008151E0+616112]
	(No symbol) [0x00725F8C]
	(No symbol) [0x00722328]
	(No symbol) [0x0072240B]
	(No symbol) [0x00714FF7]
	BaseThreadInitThunk [0x75C27D59+25]
	RtlInitializeExceptionChain [0x771FB74B+107]
	RtlClearBits [0x771FB6CF+191]


In [40]:
df.to_csv('oscar.csv')

---

In [18]:
df = pd.read_csv('oscar.csv')

wd = webdriver.Chrome(ChromeDriverManager().install())

movie_indices = np.arange(0,925,1)

for movie_index in movie_indices[5:925]:
    movie_title = df['Film'][movie_index] # get song title
    movie_year = df['Year'][movie_index] # get song title
    wd.get('https://www.wikipedia.com/') # go to main site
    search_movie = wd.find_element("xpath", "//*[@id='searchInput']") 
    search_movie.send_keys(str(movie_title)+" "+str(movie_year)+" film")
    search_movie.send_keys(Keys.RETURN)

    try:
        infobox_table = wd.find_element("xpath", "//table[@class='infobox vevent']")
        rows = infobox_table.find_elements(By.TAG_NAME, "tr")
        budget_row = None
        for row in rows:
            if "Budget" in row.text:
                budget_row = row
                break

        if budget_row is not None:
            df['Budget'][movie_index] = budget_row.find_element(By.TAG_NAME, "td").text

            if np.isnan(df['Budget'][movie_index]):
                wd.get('https://www.wikipedia.com/') # go to main site
                search_movie = wd.find_element("xpath", "//*[@id='searchInput']") 
                search_movie.send_keys(str(movie_title)+" "+str(movie_year)+" film")
                search_movie.send_keys(Keys.RETURN)   
                try:
                    hyperlink = wd.find_element("xpath", "//*[@id='mw-content-text']/div[3]/div[4]/ul/li[1]/table/tbody/tr/td[2]/div[1]/a")
                    hyperlink.send_keys(Keys.RETURN)
                    infobox_table = wd.find_element("xpath", "//table[@class='infobox vevent']")
                    rows = infobox_table.find_elements(By.TAG_NAME, "tr")
                    budget_row = None
                    for row in rows:
                        if "Budget" in row.text:
                            budget_row = row
                            break

                    if budget_row is not None:
                        df['Budget'][movie_index] = budget_row.find_element(By.TAG_NAME, "td").text
                    else:
                        pass
                except:
                    pass
            else:
                pass
        else:
            pass
    except:
        pass

C:\Users\redoc\AppData\Local\Temp\ipykernel_3780\655367756.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0078A813+48355]
	(No symbol) [0x0071C4B1]
	(No symbol) [0x00625358]
	(No symbol) [0x0060D293]
	(No symbol) [0x0066E37B]
	(No symbol) [0x0067C473]
	(No symbol) [0x0066A536]
	(No symbol) [0x006482DC]
	(No symbol) [0x006493DD]
	GetHandleVerifier [0x009EAABD+2539405]
	GetHandleVerifier [0x00A2A78F+2800735]
	GetHandleVerifier [0x00A2456C+2775612]
	GetHandleVerifier [0x008151E0+616112]
	(No symbol) [0x00725F8C]
	(No symbol) [0x00722328]
	(No symbol) [0x0072240B]
	(No symbol) [0x00714FF7]
	BaseThreadInitThunk [0x75C27D59+25]
	RtlInitializeExceptionChain [0x771FB74B+107]
	RtlClearBits [0x771FB6CF+191]


---

# Scrape from Google

In [3]:
df = pd.read_csv('oscar.csv')

wd = webdriver.Chrome(ChromeDriverManager().install())

movie_indices = np.arange(0,925,1)

for movie_index in movie_indices[33:925]:
    movie_title = df['Film'][movie_index] # get song title
    movie_year = df['Year'][movie_index] # get song title
    wd.get('https://www.google.com/') # go to imdb main site
    search_movie = wd.find_element("xpath", "//*[@id='APjFqb']") 
    search_movie.send_keys(str(movie_title)+" "+str(movie_year)+" film"+" wikipedia")
    search_movie.send_keys(Keys.RETURN)

    try:
        wikipedia_link = wd.find_element("xpath", "//*[@id='rso']/div[1]/div/div/div/div[1]/div/div/div[1]/div/a/h3")
    
        wikipedia_link.click()

        infobox_table = wd.find_element("xpath", "//table[@class='infobox vevent']")
        rows = infobox_table.find_elements(By.TAG_NAME, "tr")
        time.sleep(2)
        budget_row = None

        for row in rows:
            if "Budget" in row.text:
                budget_row = row
                break

        if budget_row is not None:
            budget = budget_row.find_element(By.TAG_NAME, "td").text
            #print("Movie budget:", budget)
            df['Budget'][movie_index] = budget_row.find_element(By.TAG_NAME, "td").text

        elif np.isnan(df['Budget'][movie_index]):
            wd.get('https://www.google.com/') # go to imdb main site
            search_movie = wd.find_element("xpath", "//*[@id='APjFqb']") 
            search_movie.send_keys(str(movie_title)+" "+str(movie_year)+" film"+" wikipedia")
            search_movie.send_keys(Keys.RETURN)
            try: 
                wikipedia_link = wd.find_element("xpath", "//*[@id='rso']/div[1]/div/div/div[1]/div/a/h3") 
    
                wikipedia_link.click()

                infobox_table = wd.find_element("xpath", "//table[@class='infobox vevent']")
                rows = infobox_table.find_elements(By.TAG_NAME, "tr")
                time.sleep(2)
                budget_row = None
                for row in rows:
                    if "Budget" in row.text:
                        budget_row = row
                        break

                if budget_row is not None:
                    budget = budget_row.find_element(By.TAG_NAME, "td").text
                    #print("Movie budget:", budget)
                    df['Budget'][movie_index] = budget_row.find_element(By.TAG_NAME, "td").text
                else:
                    pass
            except:
                pass
        else:
            pass
    except:
        pass

C:\Users\redoc\AppData\Local\Temp\ipykernel_13828\3623602859.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\redoc\AppData\Local\Temp\ipykernel_13828\3623602859.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Budget'][movie_index] = budget_row.find_element(By.TAG_NAME, "td").text


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x010BA813+48355]
	(No symbol) [0x0104C4B1]
	(No symbol) [0x00F55358]
	(No symbol) [0x00F3D293]
	(No symbol) [0x00F9E37B]
	(No symbol) [0x00FAC473]
	(No symbol) [0x00F9A536]
	(No symbol) [0x00F782DC]
	(No symbol) [0x00F793DD]
	GetHandleVerifier [0x0131AABD+2539405]
	GetHandleVerifier [0x0135A78F+2800735]
	GetHandleVerifier [0x0135456C+2775612]
	GetHandleVerifier [0x011451E0+616112]
	(No symbol) [0x01055F8C]
	(No symbol) [0x01052328]
	(No symbol) [0x0105240B]
	(No symbol) [0x01044FF7]
	BaseThreadInitThunk [0x75C27D59+25]
	RtlInitializeExceptionChain [0x771FB74B+107]
	RtlClearBits [0x771FB6CF+191]


In [27]:
df.to_csv('oscar.csv')

In [23]:
df['Budget'][7]#== np.nan

nan

In [16]:
np.isnan(df['Budget'][7])

True

---

## Scrape from IMDB

In [8]:
df = pd.read_csv('oscar_left_update_complete.csv')

wd = webdriver.Chrome(ChromeDriverManager().install())

movie_indices = np.arange(0,925,1)

for movie_index in movie_indices[755:925]:
    movie_title = df['Film'][movie_index] # get song title
    movie_year = df['Year'][movie_index]
    wd.get('https://www.imdb.com/') # go to imdb main site
    search_movie = wd.find_element("xpath", "//*[@id='suggestion-search']")
    search_movie.send_keys(movie_title) # + " " + movie_year)
    search_movie.send_keys(Keys.RETURN)
    #time.sleep(2)
    try:
        #search_movie = wd.find_element("xpath", "//*[@id='mojo-search-text-input']") # //*[@id="search-word"]        search_movie.send_keys(song_title) # key in the movie title in search box
        #time.sleep(8)
        #search_movie.send_keys(movie_title +" "+ movie_year)
        #time.sleep(2)
        wd.find_element("xpath", "//*[@id='__next']/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div/a").click()
        
        time.sleep(2)
        df['MPAA.Rating'][movie_index] = wd.find_element("xpath","//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]/a").text
        df['IMDB.Rating'][movie_index] = wd.find_element("xpath","//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]").text
        df['Duration'][movie_index] = wd.find_element("xpath","//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[3]").text
        #df['Voter.Numbers'][movie_index] = wd.find_element("xpath","//*[@id='a-page']/main/div/div[3]/div[4]/div[4]/span[2]").text
        df['Genre'][movie_index] = wd.find_element("xpath","//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[1]/div[2]/a/span").text
        df['Metascore'][movie_index] = wd.find_element("xpath","//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/ul/li[3]/a/span/span[1]/span").text
        # df.to_csv('oscar_left_update.csv')
    except:
        pass

C:\Users\redoc\AppData\Local\Temp\ipykernel_7340\2480484242.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\redoc\AppData\Local\Temp\ipykernel_7340\2480484242.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MPAA.Rating'][movie_index] = wd.find_element("xpath","//*[@id='__next']/main/div/section[1]/section/div[3]/section/section/div[2]/div[1]/ul/li[2]/a").text
C:\Users\redoc\AppData\Local\Temp\ipykernel_7340\2480484242.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IMDB.Rating'][movie_index

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0064A813+48355]
	(No symbol) [0x005DC4B1]
	(No symbol) [0x004E5358]
	(No symbol) [0x004CD293]
	(No symbol) [0x0052E37B]
	(No symbol) [0x0053C473]
	(No symbol) [0x0052A536]
	(No symbol) [0x005082DC]
	(No symbol) [0x005093DD]
	GetHandleVerifier [0x008AAABD+2539405]
	GetHandleVerifier [0x008EA78F+2800735]
	GetHandleVerifier [0x008E456C+2775612]
	GetHandleVerifier [0x006D51E0+616112]
	(No symbol) [0x005E5F8C]
	(No symbol) [0x005E2328]
	(No symbol) [0x005E240B]
	(No symbol) [0x005D4FF7]
	BaseThreadInitThunk [0x75B37D59+25]
	RtlInitializeExceptionChain [0x7731B74B+107]
	RtlClearBits [0x7731B6CF+191]


In [9]:
df.to_csv('oscar_left_update_complete.csv')

---

## Scrape from Box Office Mojo

In [30]:
df = pd.read_csv('oscar_left_update_complete.csv')

wd = webdriver.Chrome(ChromeDriverManager().install())

movie_indices = np.arange(0,925,1)

for movie_index in movie_indices[358:925]:
    movie_title = df['Film'][movie_index] # get song title
    movie_year = df['Year'][movie_index]
    wd.get('https://www.boxofficemojo.com/') # go to imdb main site
    search_movie = wd.find_element("xpath", "//*[@id='mojo-search-text-input']")
    search_movie.send_keys(movie_title) # + " " + movie_year)
    try:
        #search_movie = wd.find_element("xpath", "//*[@id='mojo-search-text-input']") # //*[@id="search-word"]        search_movie.send_keys(song_title) # key in the movie title in search box
        #time.sleep(8)
        #search_movie.send_keys(movie_title +" "+ movie_year)
        #time.sleep(2)
        search_movie.send_keys(Keys.RETURN)
        time.sleep(2)
        wd.find_element("xpath", "//*[@id='a-page']/main/div/div/div/div/div/div[2]/a").click()
        time.sleep(2)
        df['Domestic'][movie_index] = wd.find_element("xpath","//*[@id='a-page']/main/div/div[3]/div[1]/div/div[1]/span[2]/span").text
        df['Worldwide'][movie_index] = wd.find_element("xpath","//*[@id='a-page']/main/div/div[3]/div[1]/div/div[3]/span[2]/span").text
        #df['Duration'][movie_index] = wd.find_element("xpath","//*[@id='a-page']/main/div/div[3]/div[4]/div[4]/span[2]").text
        #df['Genre'][movie_index] = wd.find_element("xpath","//*[@id='a-page']/main/div/div[3]/div[4]/div[5]/span[2]").text
        df['Opening.Weekend'][movie_index] = wd.find_element("xpath","//*[@id='a-page']/main/div/div[3]/div[4]/div[2]/span[2]/a/span").text
        # df.to_csv('oscar_left_update.csv')
    except:
        pass

C:\Users\redoc\AppData\Local\Temp\ipykernel_32424\20336552.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\redoc\AppData\Local\Temp\ipykernel_32424\20336552.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Domestic'][movie_index] = wd.find_element("xpath","//*[@id='a-page']/main/div/div[3]/div[1]/div/div[1]/span[2]/span").text
C:\Users\redoc\AppData\Local\Temp\ipykernel_32424\20336552.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Worldwide'][movie_index] = wd.find_element("xpath","//*[@id=

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0081A813+48355]
	(No symbol) [0x007AC4B1]
	(No symbol) [0x006B5358]
	(No symbol) [0x0069D293]
	(No symbol) [0x006FE37B]
	(No symbol) [0x0070C473]
	(No symbol) [0x006FA536]
	(No symbol) [0x006D82DC]
	(No symbol) [0x006D93DD]
	GetHandleVerifier [0x00A7AABD+2539405]
	GetHandleVerifier [0x00ABA78F+2800735]
	GetHandleVerifier [0x00AB456C+2775612]
	GetHandleVerifier [0x008A51E0+616112]
	(No symbol) [0x007B5F8C]
	(No symbol) [0x007B2328]
	(No symbol) [0x007B240B]
	(No symbol) [0x007A4FF7]
	BaseThreadInitThunk [0x75B37D59+25]
	RtlInitializeExceptionChain [0x7731B74B+107]
	RtlClearBits [0x7731B6CF+191]


In [31]:
df.to_csv('oscar_left_update_complete.csv')

---

## Scrape from numbers.com

In [3]:
df = pd.read_csv('oscar_left_update_complete.csv')

wd = webdriver.Chrome(ChromeDriverManager().install())

movie_indices = np.arange(0,925,1)

for movie_index in movie_indices[0:925]:
    movie_title = df['Film'][movie_index] # get song title
    wd.get('https://www.the-numbers.com/') # go to imdb main site
    try:
        search_movie = wd.find_element("xpath", "//*[@id='header']/span[2]/form/input") # //*[@id="search-word"]        search_movie.send_keys(song_title) # key in the movie title in search box
        #time.sleep(8)
        search_movie.send_keys(movie_title)
        search_movie.send_keys(Keys.RETURN)
        time.sleep(2)
        try: 
            movie = wd.find_element("xpath", "//*[@id='page_filling_chart']/center/table/tbody/tr[2]/td[3]/b/a").click()
        except:
            pass
        df['Domestic'][movie_index] = wd.find_element("xpath","//*[@id='movie_finances']/tbody/tr[2]/td[2]").text
        # df.to_csv('oscar_left_update.csv')
    except:
        pass

C:\Users\redoc\AppData\Local\Temp\ipykernel_32108\13733349.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  wd = webdriver.Chrome(ChromeDriverManager().install())
C:\Users\redoc\AppData\Local\Temp\ipykernel_32108\13733349.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Domestic'][movie_index] = wd.find_element("xpath","//*[@id='movie_finances']/tbody/tr[2]/td[2]").text


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x011FA813+48355]
	(No symbol) [0x0118C4B1]
	(No symbol) [0x01095358]
	(No symbol) [0x0107D293]
	(No symbol) [0x010DE37B]
	(No symbol) [0x010EC473]
	(No symbol) [0x010DA536]
	(No symbol) [0x010B82DC]
	(No symbol) [0x010B93DD]
	GetHandleVerifier [0x0145AABD+2539405]
	GetHandleVerifier [0x0149A78F+2800735]
	GetHandleVerifier [0x0149456C+2775612]
	GetHandleVerifier [0x012851E0+616112]
	(No symbol) [0x01195F8C]
	(No symbol) [0x01192328]
	(No symbol) [0x0119240B]
	(No symbol) [0x01184FF7]
	BaseThreadInitThunk [0x75B37D59+25]
	RtlInitializeExceptionChain [0x7731B74B+107]
	RtlClearBits [0x7731B6CF+191]


In [ ]:
df.to_csv('oscar_left_update_complete.csv')

---